### **Load Environment variables from .env file**

In [1]:
from openai import AzureOpenAI
from dotenv import load_dotenv
import os
from IPython.display import display, HTML, JSON, Markdown

load_dotenv()
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
OPENAI_DEPLOYMENT_ENDPOINT = os.getenv("OPENAI_DEPLOYMENT_ENDPOINT")
OPENAI_GPT35_DEPLOYMENT_NAME = os.getenv("OPENAI_GPT35_DEPLOYMENT_NAME")

client = AzureOpenAI(
  azure_endpoint = OPENAI_DEPLOYMENT_ENDPOINT, 
  api_key=OPENAI_API_KEY,  
  api_version="2024-02-15-preview"
)

In [2]:
def call_openAI(text):
    response = client.chat.completions.create(
        model=OPENAI_GPT35_DEPLOYMENT_NAME,
        messages = text,
        temperature=0.7,
        max_tokens=800,
        top_p=0.95,
        frequency_penalty=0,
        presence_penalty=0,
        stop=None
    )

    return response.choices[0].message.content



### **Add personality to the model and start asking questions**
We call directly the Azure OpenAI API with ***ChatCompletion*** API

In [24]:
code = """
{
    "name": "PL_INVOCATION_SICUR",
    "properties": {
        "activities": [
            {
                "name": "General-Validation",
                "type": "IfCondition",
                "dependsOn": [],
                "userProperties": [],
                "userProperties": {
                    "expression": {
                        "value": "@not(empty(pipeline().parameters.consolidationDate))",
                        "type": "Expression"
                    },
                    "ifFalseActivities": [
                        {
                            "name": "check-consolidation-proc-false",
                            "type": "SetVariable",
                            "dependsOn": [],
                            "policy": {
                                "secureOutput": false,
                                "secureInput": false
                            },
                            "userProperties": [],
                            "typeProperties": {
                                "variableName": "ThrowError",
                                "value": {
                                    "value": "@if(contains(pipeline().parameters.Valid_conProc, toLower(pipeline().parameters.consolidationProc))\n, null, div(1,0))",
                                    "type": "Expression"
                                }
                            }
                        },
                        {
                            "name": "consolidation-proc-false",
                            "type": "AppendVariable",
                            "dependsOn": [
                                {
                                    "activity": "check-consolidation-proc-false",
                                    "dependencyConditions": [
                                        "Failed"
                                    ]
                                }
                            ],
                            "userProperties": [],
                            "typeProperties": {
                                "variableName": "GeneralError",
                                "value": {
                                    "value": "@concat('<br><b>consolidationProc</b>: ', toLower(pipeline().parameters.consolidationProc))",
                                    "type": "Expression"
                                }
                            }
                        },
                        {
                            "name": "Check-env_2",
                            "type": "SetVariable",
                            "dependsOn": [],
                            "policy": {
                                "secureOutput": false,
                                "secureInput": false
                            },
                            "userProperties": [],
                            "typeProperties": {
                                "variableName": "ThrowError",
                                "value": {
                                    "value": "@if(contains(pipeline().parameters.validEnv, pipeline().parameters.env)\n, null, div(1,0))",
                                    "type": "Expression"
                                }
                            }
                        },
                        {
                            "name": "Set-folderEnv_2",
                            "type": "SetVariable",
                            "dependsOn": [
                                {
                                    "activity": "Check-env_2",
                                    "dependencyConditions": [
                                        "Succeeded"
                                    ]
                                }
                            ],
                            "policy": {
                                "secureOutput": false,
                                "secureInput": false
                            },
                            "userProperties": [],
                            "typeProperties": {
                                "variableName": "folderEnv",
                                "value": {
                                    "value": "@if(contains(pipeline().parameters.folderEnv, pipeline().parameters.env)\n, concat(pipeline().parameters.env, '/'), null )",
                                    "type": "Expression"
                                }
                            }
                        },
                        {
                            "name": "Set-v_adlsControlDir_2",
                            "type": "SetVariable",
                            "dependsOn": [
                                {
                                    "activity": "Set-folderEnv_2",
                                    "dependencyConditions": [
                                        "Succeeded"
                                    ]
                                }
                            ],
                            "policy": {
                                "secureOutput": false,
                                "secureInput": false
                            },
                            "userProperties": [],
                            "typeProperties": {
                                "variableName": "v_adlsControlDir",
                                "value": {
                                    "value": "@replace(replace(pipeline().parameters.adlsControlDir, \n'{env}', variables('folderEnv')), '//', '/')",
                                    "type": "Expression"
                                }
                            }
                        },
                        {
                            "name": "Set-v_aldsDestDir_2",
                            "type": "SetVariable",
                            "dependsOn": [
                                {
                                    "activity": "Set-folderEnv_2",
                                    "dependencyConditions": [
                                        "Succeeded"
                                    ]
                                }
                            ],
                            "policy": {
                                "secureOutput": false,
                                "secureInput": false
                            },
                            "userProperties": [],
                            "typeProperties": {
                                "variableName": "v_aldsDestDir",
                                "value": {
                                    "value": "@replace(pipeline().parameters.adlsDestDir, '{env}',variables('folderEnv'))",
                                    "type": "Expression"
                                }
                            }
                        },
                        {
                            "name": "env_2",
                            "type": "AppendVariable",
                            "dependsOn": [
                                {
                                    "activity": "Check-env_2",
                                    "dependencyConditions": [
                                        "Failed"
                                    ]
                                }
                            ],
                            "userProperties": [],
                            "typeProperties": {
                                "variableName": "GeneralError",
                                "value": {
                                    "value": "@concat('<br><b>consolidationType</b>: ', pipeline().parameters.env)",
                                    "type": "Expression"
                                }
                            }
                        },
                        {
                            "name": "Set-v_STGschema_2",
                            "type": "SetVariable",
                            "dependsOn": [
                                {
                                    "activity": "Check-env_2",
                                    "dependencyConditions": [
                                        "Succeeded"
                                    ]
                                }
                            ],
                            "policy": {
                                "secureOutput": false,
                                "secureInput": false
                            },
                            "userProperties": [],
                            "typeProperties": {
                                "variableName": "v_STGschema",
                                "value": {
                                    "value": "@if(\n      equals(pipeline().parameters.env, pipeline().parameters.validEnv[1]), \n      pipeline().parameters.STGschema_pre, \n      pipeline().parameters.STGschema\n)",
                                    "type": "Expression"
                                }
                            }
                        }
                    ],
                    "ifTrueActivities": [
                        {
                            "name": "set-consol-days",
                            "type": "SetVariable",
                            "dependsOn": [],
                            "policy": {
                                "secureOutput": false,
                                "secureInput": false
                            },
                            "userProperties": [],
                            "typeProperties": {
                                "variableName": "SetConsolDays",
                                "value": {
                                    "value": "@if(not(empty(pipeline().parameters.consolidationDate)),\nif(or(\n  equals(length(coalesce(pipeline().parameters.consolidationDays,take(array(''),0))),0),\n  greaterOrEquals(0, int(pipeline().parameters.consolidationDays))),\n\n     if(equals(pipeline().parameters.consolidationType, 'D'),   pipeline().parameters.DefaultDays_D, \n\n     if(equals(pipeline().parameters.consolidationType, 'S'),   pipeline().parameters.DefaultDays_S, \n\n      pipeline().parameters.DefaultDays_M)), \n\npipeline().parameters.consolidationDays), null)",
                                    "type": "Expression"
                                }
                            }
                        },
                        {
                            "name": "consolidation-days",
                            "type": "AppendVariable",
                            "dependsOn": [
                                {
                                    "activity": "set-consol-days",
                                    "dependencyConditions": [
                                        "Failed"
                                    ]
                                }
                            ],
                            "userProperties": [],
                            "typeProperties": {
                                "variableName": "GeneralError",
                                "value": {
                                    "value": "@concat('<br><b>consolidationDays</b>: ', pipeline().parameters.consolidationDays)",
                                    "type": "Expression"
                                }
                            }
                        },
                        {
                            "name": "check-consolidation-date",
                            "type": "SetVariable",
                            "dependsOn": [],
                            "policy": {
                                "secureOutput": false,
                                "secureInput": false
                            },
                            "userProperties": [],
                            "typeProperties": {
                                "variableName": "ThrowError",
                                "value": {
                                    "value": "@if(greater(convertFromUtc(utcNow(),'GMT Standard Time'),formatDateTime(pipeline().parameters.consolidationDate, 'yyyy-MM-ddTHH:mm:ssZ'))\n, null\n, div(1,0))",
                                    "type": "Expression"
                                }
                            }
                        },
                        {
                            "name": "check-consolidation-proc",
                            "type": "SetVariable",
                            "dependsOn": [],
                            "policy": {
                                "secureOutput": false,
                                "secureInput": false
                            },
                            "userProperties": [],
                            "typeProperties": {
                                "variableName": "ThrowError",
                                "value": {
                                    "value": "@if(contains(pipeline().parameters.Valid_conProc, toLower(pipeline().parameters.consolidationProc))\n, null, div(1,0))",
                                    "type": "Expression"
                                }
                            }
                        },
                        {
                            "name": "check-consolidation-type",
                            "type": "SetVariable",
                            "dependsOn": [],
                            "policy": {
                                "secureOutput": false,
                                "secureInput": false
                            },
                            "userProperties": [],
                            "typeProperties": {
                                "variableName": "ThrowError",
                                "value": {
                                    "value": "@if(contains(pipeline().parameters.Valid_conType, pipeline().parameters.consolidationType)\n, null, div(1,0))",
                                    "type": "Expression"
                                }
                            }
                        },
                        {
                            "name": "consolidation-date",
                            "type": "AppendVariable",
                            "dependsOn": [
                                {
                                    "activity": "check-consolidation-date",
                                    "dependencyConditions": [
                                        "Failed"
                                    ]
                                }
                            ],
                            "userProperties": [],
                            "typeProperties": {
                                "variableName": "GeneralError",
                                "value": {
                                    "value": "@concat('<br><b>consolidationDate</b>: ', pipeline().parameters.consolidationDate)",
                                    "type": "Expression"
                                }
                            }
                        },
                        {
                            "name": "consolidation-proc",
                            "type": "AppendVariable",
                            "dependsOn": [
                                {
                                    "activity": "check-consolidation-proc",
                                    "dependencyConditions": [
                                        "Failed"
                                    ]
                                }
                            ],
                            "userProperties": [],
                            "typeProperties": {
                                "variableName": "GeneralError",
                                "value": {
                                    "value": "@concat('<br><b>consolidationProc</b>: ', toLower(pipeline().parameters.consolidationProc))",
                                    "type": "Expression"
                                }
                            }
                        },
                        {
                            "name": "consolidation-type",
                            "type": "AppendVariable",
                            "dependsOn": [
                                {
                                    "activity": "check-consolidation-type",
                                    "dependencyConditions": [
                                        "Failed"
                                    ]
                                }
                            ],
                            "userProperties": [],
                            "typeProperties": {
                                "variableName": "GeneralError",
                                "value": {
                                    "value": "@concat('<br><b>consolidationType</b>: ', pipeline().parameters.consolidationType)",
                                    "type": "Expression"
                                }
                            }
                        },
                        {
                            "name": "Check-env",
                            "type": "SetVariable",
                            "dependsOn": [],
                            "policy": {
                                "secureOutput": false,
                                "secureInput": false
                            },
                            "userProperties": [],
                            "typeProperties": {
                                "variableName": "ThrowError",
                                "value": {
                                    "value": "@if(contains(pipeline().parameters.validEnv, pipeline().parameters.env)\n, null, div(1,0))",
                                    "type": "Expression"
                                }
                            }
                        },
                        {
                            "name": "Set-folderEnv",
                            "type": "SetVariable",
                            "dependsOn": [
                                {
                                    "activity": "Check-env",
                                    "dependencyConditions": [
                                        "Succeeded"
                                    ]
                                }
                            ],
                            "policy": {
                                "secureOutput": false,
                                "secureInput": false
                            },
                            "userProperties": [],
                            "typeProperties": {
                                "variableName": "folderEnv",
                                "value": {
                                    "value": "@if(contains(pipeline().parameters.folderEnv, pipeline().parameters.env)\n, concat(pipeline().parameters.env, '/'), null )",
                                    "type": "Expression"
                                }
                            }
                        },
                        {
                            "name": "Set-v_adlsControlDir",
                            "type": "SetVariable",
                            "dependsOn": [
                                {
                                    "activity": "Set-folderEnv",
                                    "dependencyConditions": [
                                        "Succeeded"
                                    ]
                                }
                            ],
                            "policy": {
                                "secureOutput": false,
                                "secureInput": false
                            },
                            "userProperties": [],
                            "typeProperties": {
                                "variableName": "v_adlsControlDir",
                                "value": {
                                    "value": "@replace(replace(pipeline().parameters.adlsControlDir, \n'{env}', variables('folderEnv')), '//', '/')",
                                    "type": "Expression"
                                }
                            }
                        },
                        {
                            "name": "Set-v_aldsDestDir",
                            "type": "SetVariable",
                            "dependsOn": [
                                {
                                    "activity": "Set-folderEnv",
                                    "dependencyConditions": [
                                        "Succeeded"
                                    ]
                                }
                            ],
                            "policy": {
                                "secureOutput": false,
                                "secureInput": false
                            },
                            "userProperties": [],
                            "typeProperties": {
                                "variableName": "v_aldsDestDir",
                                "value": {
                                    "value": "@replace(pipeline().parameters.adlsDestDir, '{env}',variables('folderEnv'))",
                                    "type": "Expression"
                                }
                            }
                        },
                        {
                            "name": "env",
                            "type": "AppendVariable",
                            "dependsOn": [
                                {
                                    "activity": "Check-env",
                                    "dependencyConditions": [
                                        "Failed"
                                    ]
                                }
                            ],
                            "userProperties": [],
                            "typeProperties": {
                                "variableName": "GeneralError",
                                "value": {
                                    "value": "@concat('<br><b>consolidationType</b>: ', pipeline().parameters.env)",
                                    "type": "Expression"
                                }
                            }
                        },
                        {
                            "name": "Set-v_STGschema",
                            "type": "SetVariable",
                            "dependsOn": [
                                {
                                    "activity": "Check-env",
                                    "dependencyConditions": [
                                        "Succeeded"
                                    ]
                                }
                            ],
                            "policy": {
                                "secureOutput": false,
                                "secureInput": false
                            },
                            "userProperties": [],
                            "typeProperties": {
                                "variableName": "v_STGschema",
                                "value": {
                                    "value": "@if(\n      equals(pipeline().parameters.env, pipeline().parameters.validEnv[1]), \n      pipeline().parameters.STGschema_pre, \n      pipeline().parameters.STGschema\n)",
                                    "type": "Expression"
                                }
                            }
                        }
                    ]
                }
            },
            {
                "name": "Import-Table",
                "type": "IfCondition",
                "dependsOn": [
                    {
                        "activity": "If-Error",
                        "dependencyConditions": [
                            "Succeeded"
                        ]
                    }
                ],
                "userProperties": [],
                "typeProperties": {
                    "expression": {
                        "value": "@not(empty(pipeline().parameters.tables))",
                        "type": "Expression"
                    },
                    "ifTrueActivities": [
                        {
                            "name": "PL_SETUP_IMPORT_To_DS_ADLS",
                            "type": "ExecutePipeline",
                            "dependsOn": [
                                {
                                    "activity": "STG-validSchema",
                                    "dependencyConditions": [
                                        "Succeeded"
                                    ]
                                }
                            ],
                            "policy": {
                                "secureInput": false
                            },
                            "userProperties": [],
                            "typeProperties": {
                                "pipeline": {
                                    "referenceName": "PL_SETUP_IMPORT_To_DS_ADLS_SICUR",
                                    "type": "PipelineReference"
                                },
                                "waitOnCompletion": true,
                                "parameters": {
                                    "consolidationDate": {
                                        "value": "@pipeline().parameters.consolidationDate",
                                        "type": "Expression"
                                    },
                                    "tables": {
                                        "value": "@pipeline().parameters.tables",
                                        "type": "Expression"
                                    },
                                    "consolidationProc": {
                                        "value": "@toLower(pipeline().parameters.consolidationProc)",
                                        "type": "Expression"
                                    },
                                    "adlsDestinationDir": {
                                        "value": "@variables('v_aldsDestDir')",
                                        "type": "Expression"
                                    },
                                    "nrRows": {
                                        "value": "@pipeline().parameters.nrRows",
                                        "type": "Expression"
                                    },
                                    "env": {
                                        "value": "@pipeline().parameters.env",
                                        "type": "Expression"
                                    },
                                    "STGschema": {
                                        "value": "@variables('v_STGschema')",
                                        "type": "Expression"
                                    },
                                    "adlsControlDir": {
                                        "value": "@variables('v_adlsControlDir')",
                                        "type": "Expression"
                                    },
                                    "adlsDestinationContainer": {
                                        "value": "@pipeline().parameters.adlsContainer",
                                        "type": "Expression"
                                    },
                                    "parentPipeline": {
                                        "value": "@pipeline().RunId",
                                        "type": "Expression"
                                    },
                                    "Valid_tabSchemas": {
                                        "value": "@variables('validTabSchemasImport')",
                                        "type": "Expression"
                                    },
                                    "idInput": {
                                        "value": "@pipeline().parameters.idInput",
                                        "type": "Expression"
                                    }
                                }
                            }
                        },
                        {
                            "name": "STG-validSchema",
                            "type": "AppendVariable",
                            "dependsOn": [],
                            "userProperties": [],
                            "typeProperties": {
                                "variableName": "validTabSchemasImport",
                                "value": {
                                    "value": "@variables('v_STGschema')",
                                    "type": "Expression"
                                }
                            }
                        }
                    ]
                }
            },
            {
                "name": "If-Error",
                "type": "IfCondition",
                "dependsOn": [
                    {
                        "activity": "General-Validation",
                        "dependencyConditions": [
                            "Succeeded"
                        ]
                    }
                ],
                "userProperties": [],
                "typeProperties": {
                    "expression": {
                        "value": "@not(empty(variables('GeneralError')))",
                        "type": "Expression"
                    },
                    "ifTrueActivities": [
                        {
                            "name": "Write-Error-Validation-Dbricks",
                            "type": "Copy",
                            "dependsOn": [],
                            "policy": {
                                "timeout": "7.00:00:00",
                                "retry": 0,
                                "retryIntervalInSeconds": 30,
                                "secureOutput": false,
                                "secureInput": false
                            },
                            "userProperties": [],
                            "typeProperties": {
                                "source": {
                                    "type": "OracleSource",
                                    "oracleReaderQuery": {
                                        "value": "@concat('SELECT ''Erro durante a validação dos seguintes parâmetros de invocação do processo databricks. Por favor valide os parâmetros antes de efetuar nova invocação: ',\nreplace(replace(replace(string(variables('GeneralError')),']', ''),'[', ''), '\"', ''), ''' AS ErrorThrown, \n''ERROR'' AS Status,\n''U_S'' AS Operation, ''', \npipeline().parameters.env, ''' AS env,  ''', \npipeline().RunId, ''' AS parentPipeline \nFROM DUAL')",
                                        "type": "Expression"
                                    },
                                    "partitionOption": "None",
                                    "queryTimeout": "02:00:00"
                                },
                                "sink": {
                                    "type": "JsonSink",
                                    "storeSettings": {
                                        "type": "AzureBlobFSWriteSettings"
                                    },
                                    "formatSettings": {
                                        "type": "JsonWriteSettings"
                                    }
                                },
                                "enableStaging": false,
                                "translator": {
                                    "type": "TabularTranslator",
                                    "mappings": [
                                        {
                                            "source": {
                                                "name": "ERRORTHROWN"
                                            },
                                            "sink": {
                                                "path": "$['errorThrown']"
                                            }
                                        },
                                        {
                                            "source": {
                                                "name": "OPERATION"
                                            },
                                            "sink": {
                                                "path": "$['operation']"
                                            }
                                        },
                                        {
                                            "source": {
                                                "name": "STATUS"
                                            },
                                            "sink": {
                                                "path": "$['status']"
                                            }
                                        },
                                        {
                                            "source": {
                                                "name": "PARENTPIPELINE"
                                            },
                                            "sink": {
                                                "path": "$['parentPipeline']"
                                            }
                                        }
                                    ]
                                }
                            },
                            "inputs": [
                                {
                                    "referenceName": "DS_ORAC_PWC2PDB1",
                                    "type": "DatasetReference"
                                }
                            ],
                            "outputs": [
                                {
                                    "referenceName": "DS_ADLS_JSON",
                                    "type": "DatasetReference",
                                    "parameters": {
                                        "container": {
                                            "value": "@pipeline().parameters.adlsContainer",
                                            "type": "Expression"
                                        },
                                        "dir": {
                                            "value": "@concat(variables('v_adlsControlDir'),'trigger/error')",
                                            "type": "Expression"
                                        },
                                        "file": {
                                            "value": "@concat(pipeline().RunId, '.json')",
                                            "type": "Expression"
                                        }
                                    }
                                }
                            ]
                        },
                        {
                            "name": "throw-error-validation-dbricks",
                            "type": "SetVariable",
                            "dependsOn": [
                                {
                                    "activity": "Write-Error-Validation-Dbricks",
                                    "dependencyConditions": [
                                        "Succeeded"
                                    ]
                                }
                            ],
                            "policy": {
                                "secureOutput": false,
                                "secureInput": false
                            },
                            "userProperties": [],
                            "typeProperties": {
                                "variableName": "ThrowError",
                                "value": {
                                    "value": "@div(1,0)",
                                    "type": "Expression"
                                }
                            }
                        }
                    ]
                }
            },
            {
                "name": "Write-Trg-Pre",
                "type": "Copy",
                "dependsOn": [
                    {
                        "activity": "Import-Table",
                        "dependencyConditions": [
                            "Succeeded"
                        ]
                    }
                ],
                "policy": {
                    "timeout": "7.00:00:00",
                    "retry": 0,
                    "retryIntervalInSeconds": 30,
                    "secureOutput": false,
                    "secureInput": false
                },
                "userProperties": [],
                "typeProperties": {
                    "source": {
                        "type": "OracleSource",
                        "oracleReaderQuery": {
                            "value": "@concat('SELECT ''',\npipeline().RunId, ''' AS parentPipeline, \n''', pipeline().Pipeline, ''' AS pipelineName, \n''', pipeline().DataFactory, ''' AS dataFactory, \n''', string(pipeline().parameters.tables), ''' AS tables, \n''', string(pipeline().parameters.idInput), ''' AS idInput, \n''', if(\n\tempty(pipeline().parameters.consolidationDate),\n\tpipeline().parameters.consolidationDate,\n\tif(equals(length(formatDateTime(pipeline().parameters.consolidationDate,'yyyy-MM-ddTHH:mm:ss.ffZ')),23), \n\t   replace(string(formatDateTime(pipeline().parameters.consolidationDate,'yyyy-MM-ddTHH:mm:ss.ffZ')), '.00Z', '.01Z'),\n\t   formatDateTime(pipeline().parameters.consolidationDate,'yyyy-MM-ddTHH:mm:ss.ffZ')\n\t)\n\t), ''' AS consolidationDate, \n''', pipeline().parameters.consolidationType, ''' AS consolidationType, \n''', pipeline().parameters.consolidationDays, ''' AS consolidationDays, \n''', toLower(pipeline().parameters.consolidationProc), ''' AS consolidationProc, \n''', pipeline().parameters.retryNumber, ''' AS retryNumber, \n''', pipeline().parameters.maxRetry, ''' AS maxRetry, \n''', pipeline().parameters.env, ''' AS env, \n''', variables('v_aldsDestDir'), ''' AS stgInputDir, \n''', pipeline().parameters.OGGschema, ''' AS OGGschema, \n''', variables('v_STGschema'), ''' AS STGschema, \n''', if(not(empty(pipeline().parameters.tables)), pipeline().parameters.consolidationProc, null), ''' AS isImporting \nFROM DUAL')",
                            "type": "Expression"
                        },
                        "partitionOption": "None",
                        "queryTimeout": "02:00:00"
                    },
                    "sink": {
                        "type": "JsonSink",
                        "storeSettings": {
                            "type": "AzureBlobFSWriteSettings"
                        },
                        "formatSettings": {
                            "type": "JsonWriteSettings"
                        }
                    },
                    "enableStaging": false,
                    "translator": {
                        "type": "TabularTranslator",
                        "mappings": [
                            {
                                "source": {
                                    "name": "PARENTPIPELINE"
                                },
                                "sink": {
                                    "path": "$['parentPipeline']"
                                }
                            },
                            {
                                "source": {
                                    "name": "PIPELINENAME"
                                },
                                "sink": {
                                    "path": "$['pipelineName']"
                                }
                            },
                            {
                                "source": {
                                    "name": "TABLES"
                                },
                                "sink": {
                                    "path": "$['tables']"
                                }
                            },
                            {
                                "source": {
                                    "name": "IDINPUT"
                                },
                                "sink": {
                                    "path": "$['idInput']"
                                }
                            },
                            {
                                "source": {
                                    "name": "CONSOLIDATIONDATE"
                                },
                                "sink": {
                                    "path": "$['consolidationDate']"
                                }
                            },
                            {
                                "source": {
                                    "name": "CONSOLIDATIONTYPE"
                                },
                                "sink": {
                                    "path": "$['consolidationType']"
                                }
                            },
                            {
                                "source": {
                                    "name": "CONSOLIDATIONDAYS"
                                },
                                "sink": {
                                    "path": "$['consolidationDays']"
                                }
                            },
                            {
                                "source": {
                                    "name": "CONSOLIDATIONPROC"
                                },
                                "sink": {
                                    "path": "$['consolidationProc']"
                                }
                            },
                            {
                                "source": {
                                    "name": "RETRYNUMBER"
                                },
                                "sink": {
                                    "path": "$['retryNumber']"
                                }
                            },
                            {
                                "source": {
                                    "name": "MAXRETRY"
                                },
                                "sink": {
                                    "path": "$['maxRetry']"
                                }
                            },
                            {
                                "source": {
                                    "name": "ISIMPORTING"
                                },
                                "sink": {
                                    "path": "$['isImporting']"
                                }
                            },
                            {
                                "source": {
                                    "name": "DATAFACTORY"
                                },
                                "sink": {
                                    "path": "$['dataFactory']"
                                }
                            },
                            {
                                "source": {
                                    "name": "OGGSCHEMA"
                                },
                                "sink": {
                                    "path": "$['OGGschema']"
                                }
                            },
                            {
                                "source": {
                                    "name": "STGSCHEMA"
                                },
                                "sink": {
                                    "path": "$['STGschema']"
                                }
                            },
                            {
                                "source": {
                                    "name": "ENV"
                                },
                                "sink": {
                                    "path": "$['env']"
                                }
                            },
                            {
                                "source": {
                                    "name": "STGINPUTDIR"
                                },
                                "sink": {
                                    "path": "$['stgInputDir']"
                                }
                            }
                        ]
                    }
                },
                "inputs": [
                    {
                        "referenceName": "DS_ORAC_PWC2PDB1",
                        "type": "DatasetReference"
                    }
                ],
                "outputs": [
                    {
                        "referenceName": "DS_ADLS_JSON",
                        "type": "DatasetReference",
                        "parameters": {
                            "container": {
                                "value": "@pipeline().parameters.adlsContainer",
                                "type": "Expression"
                            },
                            "dir": {
                                "value": "@concat(variables('v_adlsControlDir'),'trigger/dbricks')",
                                "type": "Expression"
                            },
                            "file": {
                                "value": "@concat(pipeline().RunId, '.json')",
                                "type": "Expression"
                            }
                        }
                    }
                ]
            }
        ],
        "parameters": {
            "nrRows": {
                "type": "string"
            },
            "retryNumber": {
                "type": "string",
                "defaultValue": "0"
            },
            "STGschema_pre": {
                "type": "string",
                "defaultValue": "STG_MDUSU_SU"
            },
            "Valid_conType": {
                "type": "array",
                "defaultValue": [
                    "D",
                    "S",
                    "M"
                ]
            },
            "adlsControlDir": {
                "type": "string",
                "defaultValue": "data/control/adf/{env}/invocation/"
            },
            "DefaultDays_S": {
                "type": "string",
                "defaultValue": "7"
            },
            "maxRetry": {
                "type": "string",
                "defaultValue": "2"
            },
            "tables": {
                "type": "array",
                "defaultValue": []
            },
            "folderEnv": {
                "type": "array",
                "defaultValue": [
                    "dev",
                    "pre"
                ]
            },
            "adlsContainer": {
                "type": "string",
                "defaultValue": "data"
            },
            "consolidationProc": {
                "type": "string",
                "defaultValue": "pgsus"
            },
            "validEnv": {
                "type": "array",
                "defaultValue": [
                    "dev",
                    "pre",
                    "prd"
                ]
            },
            "consolidationDate": {
                "type": "string",
                "defaultValue": "2021-08-06T09:34:15.02Z"
            },
            "OggProcesses": {
                "type": "string"
            },
            "DefaultDays_D": {
                "type": "string",
                "defaultValue": "2"
            },
            "consolidationType": {
                "type": "string",
                "defaultValue": "D"
            },
            "env": {
                "type": "string",
                "defaultValue": "prd"
            },
            "consolidationDays": {
                "type": "string",
                "defaultValue": "500"
            },
            "STGschema": {
                "type": "string",
                "defaultValue": "STG_MDUSU_SU"
            },
            "DefaultDays_M": {
                "type": "string",
                "defaultValue": "31"
            },
            "Valid_conProc": {
                "type": "array",
                "defaultValue": [
                    "ogg",
                    "stg",
                    "pgsus"
                ]
            },
            "idInput": {
                "type": "array"
            },
            "OGGschema": {
                "type": "string",
                "defaultValue": "SAPSR3"
            },
            "adlsDestDir": {
                "type": "string",
                "defaultValue": "/data/src/pwc_etl/{env}"
            }
        },
        "variables": {
            "folderEnv": {
                "type": "String"
            },
            "v_STGschema": {
                "type": "String"
            },
            "ThrowError": {
                "type": "String"
            },
            "v_adlsControlDir": {
                "type": "String"
            },
            "v_aldsDestDir": {
                "type": "String"
            },
            "validTabSchemasImport": {
                "type": "Array"
            },
            "GeneralError": {
                "type": "Array"
            },
            "SetConsolDays": {
                "type": "String"
            }
        },
        "folder": {
            "name": "Invocation_SICUR"
        },
        "annotations": [],
        "lastPublishTime": "2021-11-04T12:05:39Z"
    },
    "type": "Microsoft.DataFactory/factories/pipelines"
}

""" 

In [25]:
# prepare prompt
messages = [
    {
    "role": "system",
    "content": """
    You are an AI Assistant specialized in json files which represent Azure Data Factory pipelines.
    You are able to read json file content and extract tecnical information crucial for analysis by a Senior Azure Cloud Engineer and specialist in Azure Data Factory.
    You always write all information in portuguese from Portugal, but Azure component name or type are kept as is, in English.
    You are very organized, technically detailed, and clear, but concise.
    """
    },
    {
    "role": "user",
    "content": f"""
    Listar as etapas ("activities") do pipeline, suas dependências ("dependsOn") e sequência no fluxo e se possui sintaxe para esperar os passos anteriores 
    Listar "userProperties" e interpretar possíveis "expression" existentes para cada "activities"
    Listar quais variáveis tem seus valore atribuídos pelo type "SetVariable" e com qual valor ou expressão analisada funcionalmente.
    Listar quais "parameters" tem seus valore atribuídos e em caso de expressão, que seja analisada funcionalmente.
    Here is the json file content below:
    {code}
    """
    }
]
 
info_listed = call_openAI(messages)
display(HTML(info_listed))

In [26]:
# prepare prompt
messages = [
    {
    "role": "system",
    "content": """
    You are an AI Assistant specialized in json files which represent Azure Data Factory pipelines.
    You are able to read json file content and extract tecnical information crucial for analysis by a Senior Azure Cloud Engineer and specialist in Azure Data Factory.
    You always write all information in portuguese from Portugal, but Azure component name or type are kept as is, in English.
    You are very organized, technically detailed, and clear, but concise.
    """
    },
    {
    "role": "user",
    "content": f"""
    Para os type "Copy", mapear pontualmente seus "typeProperties" ("source", "sink" e "PARENTPIPELINE") com seus respetivos "type" ("OracleSource", "JsonSink", etc), identificando claramente os tipos de formato dos dados, tais como de banco de dados, ficheiros json, csv, txt, etc.
    Adcionalmente, para cada "Copy", Listar as informações de "timeout" (ex: "7.00:00:00"), "retry" e "retryIntervalInSeconds" para que sejam adicionados a cada processo de Copy mapeado.
    Mapear seus "inputs" e "outputs" de dados com seus respetivos type (ex: "DatasetReference") e "parameters", tais como: "container", "dir" e "file", interpretados funcionalmente caso possuam "Expression"
    Por fim, analise cada "translator", documentando o "mappings" para cada par "source" / "sink" com seus respetivos valores ou expressões documentados funcionalmente
    Here is the json file content below:
    {code}
    """
    }
]
 
info_analysed = call_openAI(messages)
display(HTML(info_analysed))

In [28]:
# prepare prompt
messages = [
    {
    "role": "system",
    "content": """
    You are Senior Azure Cloud Engineer and specialist in Azure Data Factory.
    You are able to create professional technical documents writen in HTML language.
    You are very well organized, creating a document with titles, subtiles, sections and paragraphs.
    You follow the most professional documents stardards, creating indexes, tables, titles and content structure for this purpose.
    You always write all information in portuguese from Portugal, but Azure component name or type are kept as is, in English.
    You are very organized, technically detailed, and clear, but concise.
    """
    },
    {
    "role": "user",
    "content": f"""
    Grap techncial information below to generate final content to the document

    Here is the info listed by the AI assistant :
    {info_listed}

    Here is the info analysed by the AI assistant :
    {info_analysed}

    Create HTML file based on document content, formating it properly with proper HTML tags and present the final HTML file.
    """

    }
]
 
document = call_openAI(messages)
display(HTML(document))

In [ ]:
messages = [
    {
    "role": "system",
    "content": """
    Your firt action will be to interpret an Azure Data Factory pipeline content exported in json format.
    Secondly, you will grap techncial information about triggers, variables names, data sources, sinks and main flow steps
    Afterwards, you will interpret logical expressions in variables or input fields and adjust the information to be funcionally understandable
    Identify any error handling step and behaviour and Integration runtimes usage, with all infrastructure information available for further usage
    Finally, interpret logical flows and document everything in a structure way.
    """
    }
]

In [ ]:
# prepare prompt with another question:
messages = [{"role": "system", "content": "You are a HELPFUL assistant answering users trivia questions. Answer in clear and concise manner."},
            {"role": "user", "content": "What's string theory?"}]

result = call_openAI(messages)
display(HTML(result))

In [ ]:
# prompt with specific instructions
messages = [{"role": "system", "content": "You are a HELPFUL assistant answering users trivia questions. Answer as for a FIVE YEARS old child."},
            {"role": "user", "content": "what's string theory?"}]


result = call_openAI(messages)
display(HTML(result))

In [ ]:
# prompt with specific instructions
messages = [{"role": "system", "content": "You are a HELPFUL assistant that knows Python"},
            {"role": "user", "content": "Create a Python function that takes a string argument and reverses it."}]


result = call_openAI(messages)
display(Markdown(result))